In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/p1_train.csv')
df.head()

,f00,f01,f02,f03,f04,f05,f06,f07,f08,f09,...,f22,f23,f24,f25,f26,f27,f28,f29,f30,class
0,33.682563,9.025679,21.463532,242.366840,0,1.725883,6.327112,13.297218,73,0,...,1,1,2,0,1,0,0,1.347214,0,QqQqQ
1,79.028477,7.151293,20.613267,231.162595,0,2.592424,7.619885,4.542524,89,0,...,0,0,0,0,0,0,0,0.518767,0,QqQqQ
2,68.457491,8.392554,13.991127,159.582240,0,6.481226,8.428001,12.209266,74,0,...,0,0,1,0,0,0,0,7.435604,1,QqQqQ
3,56.874305,5.597238,13.517609,237.602184,0,0.014691,6.310461,18.454356,89,0,...,0,1,0,1,0,0,0,0.795498,0,QqQqQ
4,66.533369,5.477686,1.964413,263.734149,0,9.236328,9.257695,0.646047,68,0,...,0,0,2,0,0,0,3,5.897388,0,QqQqQ


In [3]:
df.shape

(1531, 32)

In [4]:
X, Y = df.drop(['class'], axis=1), df['class']

In [5]:
X.head()

,f00,f01,f02,f03,f04,f05,f06,f07,f08,f09,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
0,33.682563,9.025679,21.463532,242.366840,0,1.725883,6.327112,13.297218,73,0,...,72,1,1,2,0,1,0,0,1.347214,0
1,79.028477,7.151293,20.613267,231.162595,0,2.592424,7.619885,4.542524,89,0,...,64,0,0,0,0,0,0,0,0.518767,0
2,68.457491,8.392554,13.991127,159.582240,0,6.481226,8.428001,12.209266,74,0,...,115,0,0,1,0,0,0,0,7.435604,1
3,56.874305,5.597238,13.517609,237.602184,0,0.014691,6.310461,18.454356,89,0,...,117,0,1,0,1,0,0,0,0.795498,0
4,66.533369,5.477686,1.964413,263.734149,0,9.236328,9.257695,0.646047,68,0,...,88,0,0,2,0,0,0,3,5.897388,0


In [6]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [7]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X.shape

(1531, 31)

In [8]:
Y.head()

0    QqQqQ
1    QqQqQ
2    QqQqQ
3    QqQqQ
4    QqQqQ
Name: class, dtype: object

In [9]:
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y

array([1, 1, 1, ..., 0, 0, 1])

In [10]:
Y.shape, Y.sum()

((1531,), np.int64(987))

In [11]:
import keras

In [12]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(X.shape[-1],)))
model.add(keras.layers.Dense(units=16, activation='relu'))
model.add(keras.layers.Dense(units=1, activation='sigmoid'))
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 529 (2.07 KB)

 Trainable params: 529 (2.07 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

In [14]:
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])

In [15]:
model.fit(X, Y, epochs=100, batch_size=Y.shape[0], shuffle=True, verbose=0)

In [16]:
df_test = pd.read_csv('data/p1_test_student.csv')
X_test, Y_test = df_test.drop(['class'], axis=1), df_test['class']
X_test = scaler.transform(X_test)
Y_predict = model.predict(X_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [17]:
Y_test = encoder.transform(Y_test).reshape(-1, 1)
Y_test.shape

(305, 1)

In [18]:
Y_predict = (Y_predict >= 0.5).astype(int)
Y_predict.shape

(305, 1)

In [19]:
accuracy = (Y_predict == Y_test).sum() / Y_test.shape[0]
accuracy

np.float64(0.7672131147540984)

In [20]:
(Y_predict == 1).sum(), (Y_predict == 0).sum()

(np.int64(223), np.int64(82))

In [21]:
(Y_test == 1).sum(), (Y_test == 0).sum()

(np.int64(196), np.int64(109))

In [22]:
TP = ((Y_predict == 1) & (Y_test == 1)).sum()
TP

np.int64(174)

In [23]:
FP = ((Y_predict == 1) & (Y_test != 1)).sum()
FP

np.int64(49)

In [24]:
precision = TP / (TP + FP)
precision

np.float64(0.7802690582959642)

In [25]:
FN = ((Y_predict != 1) & (Y_test == 0)).sum()
FN

np.int64(60)

In [26]:
recall =  TP / (TP + FN)
recall

np.float64(0.7435897435897436)

In [27]:
f1_score = 2 * precision * recall / (precision + recall)
f1_score

np.float64(0.761487964989059)